In [42]:
import numpy as np
import cv2
import os
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, LSTM, Dense, Input, TimeDistributed
from tensorflow.keras.models import Model
from tensorflow.keras.models import Sequential, load_model
from sklearn.metrics import accuracy_score, classification_report, roc_auc_score

In [29]:
def extract_frames_from_video(video_path, frame_count=5):
    cap = cv2.VideoCapture(video_path)
    frames = []
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    step = max(total_frames // frame_count, 1)

    for i in range(0, total_frames, step):
        cap.set(cv2.CAP_PROP_POS_FRAMES, i)
        ret, frame = cap.read()
        if ret:
            frames.append(frame)
        if len(frames) >= frame_count:
            break
    
    cap.release()
    while len(frames) < frame_count:
        frames.append(frames[-1])  # Pad with the last frame if needed
    
    return frames

def load_video_dataset(data_dir, frame_count=5):
    X, y = [], []
    class_labels = {'YouTube-real': 0, 'Celeb-synthesis': 1, 'Celeb-real': 0}
    
    for label, value in class_labels.items():
        subdir = os.path.join(data_dir, label)
        for file in os.listdir(subdir):
            if file.endswith('.mp4'):
                video_path = os.path.join(subdir, file)
                frames = extract_frames_from_video(video_path, frame_count=frame_count)
                X.append(frames)
                y.append(value)
    
    return X, y

In [30]:
def apply_sobel_filter(image, target_size=(256, 256)):
    try:
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=5)
        sobely = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=5)
        sobel = cv2.magnitude(sobelx, sobely)
        sobel_resized = cv2.resize(sobel, target_size)
        return sobel_resized
    except Exception as e:
        print(f"Error applying Sobel filter: {e}")
        return None

In [31]:

def process_frames(X, target_size=(256, 256)):
    X_processed = []
    for video in X:
        processed_video = []
        for frame in video:
            processed_frame = apply_sobel_filter(frame, target_size)
            if processed_frame is not None:
                processed_video.append(processed_frame)
        if len(processed_video) == len(video):
            X_processed.append(processed_video)
    return np.array(X_processed)

In [32]:
def build_cnn_rnn_model(input_shape, num_classes):
    inputs = Input(shape=input_shape)
    x = TimeDistributed(Conv2D(32, (3, 3), activation='relu', padding='same'))(inputs)
    x = TimeDistributed(MaxPooling2D((2, 2)))(x)
    x = TimeDistributed(Conv2D(64, (3, 3), activation='relu', padding='same'))(x)
    x = TimeDistributed(MaxPooling2D((2, 2)))(x)
    x = TimeDistributed(Flatten())(x)
    x = LSTM(128, return_sequences=False)(x)
    x = Dense(num_classes, activation='softmax')(x)
    
    model = Model(inputs=inputs, outputs=x)
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

In [33]:
frame_count = 5
target_size = (256, 256)
num_classes = 2
data_dir = 'Celeb-DF'

In [34]:
X, y = load_video_dataset(data_dir, frame_count=frame_count)
X_processed = process_frames(X, target_size=target_size)
X_processed = X_processed / 255.0
X_processed = np.expand_dims(X_processed, axis=-1)

In [35]:
X_train, X_test, y_train, y_test = train_test_split(X_processed, np.array(y), test_size=0.2, random_state=42)

In [36]:
print(f"Shape of X_train: {X_train.shape}")
print(f"Shape of y_train: {np.array(y_train).shape}")
print(f"Type of X_train: {type(X_train)}")
print(f"Type of y_train: {type(y_train)}")

Shape of X_train: (962, 5, 256, 256, 1)
Shape of y_train: (962,)
Type of X_train: <class 'numpy.ndarray'>
Type of y_train: <class 'numpy.ndarray'>


In [37]:
if len(X_train.shape) == 4:
    X_train = np.expand_dims(X_train, axis=1)  # Adding the sequence dimension
    X_test = np.expand_dims(X_test, axis=1)

input_shape = (frame_count, target_size[0], target_size[1], 1)

In [38]:
# Build and train the model
cnn_rnn_model = build_cnn_rnn_model(input_shape, num_classes)
cnn_rnn_model.fit(X_train, y_train, epochs=10, batch_size=4, validation_data=(X_test, y_test))

Epoch 1/10
241/241 ━━━━━━━━━━━━━━━━━━━━ 439s 2s/step - accuracy: 0.4347 - loss: 1.6365 - val_accuracy: 0.7593 - val_loss: 0.5839
Epoch 2/10
241/241 ━━━━━━━━━━━━━━━━━━━━ 420s 2s/step - accuracy: 0.7240 - loss: 0.5697 - val_accuracy: 0.8631 - val_loss: 0.3756
Epoch 3/10
241/241 ━━━━━━━━━━━━━━━━━━━━ 417s 2s/step - accuracy: 0.8008 - loss: 0.4726 - val_accuracy: 0.8672 - val_loss: 0.3962
Epoch 4/10
241/241 ━━━━━━━━━━━━━━━━━━━━ 417s 2s/step - accuracy: 0.8144 - loss: 0.4587 - val_accuracy: 0.8631 - val_loss: 0.3838
Epoch 5/10
241/241 ━━━━━━━━━━━━━━━━━━━━ 418s 2s/step - accuracy: 0.8377 - loss: 0.4096 - val_accuracy: 0.8506 - val_loss: 0.4230
Epoch 6/10
241/241 ━━━━━━━━━━━━━━━━━━━━ 413s 2s/step - accuracy: 0.8344 - loss: 0.4246 - val_accuracy: 0.8589 - val_loss: 0.3918
Epoch 7/10
241/241 ━━━━━━━━━━━━━━━━━━━━ 417s 2s/step - accuracy: 0.8247 - loss: 0.4230 - val_accuracy: 0.8589 - val_loss: 0.3876
Epoch 8/10
241/241 ━━━━━━━━━━━━━━━━━━━━ 417s 2s/step - accuracy: 0.8108 - loss: 0.4485 - val_accu

In [39]:
cnn_rnn_model.save('Model/cnn_rnn_model.h5')

In [49]:
def evaluate_model(model_path, X_test, y_test, class_names=['Class 0', 'Class 1']):
    """
    Evaluate a loaded model in terms of accuracy, ROC-AUC, and classification report.
    
    :param model_path: Path to the model file.
    :param X_test: Test data.
    :param y_test: Test labels.
    :param class_names: Names of the classes for the classification report.
    :return: Tuple containing accuracy, ROC-AUC, and the classification report.
    """
    # Load the model
    model = load_model(model_path)
    
    # Make predictions
    y_pred_prob = model.predict(X_test)
    
    # Convert probabilities to binary predictions
    y_pred = (y_pred_prob > 0.5).astype(int)
    
    # If y_pred is one-hot encoded, convert it to a binary vector
    if y_pred.ndim > 1 and y_pred.shape[1] > 1:
        y_pred = np.argmax(y_pred, axis=1)
    
    # Calculate accuracy
    accuracy = accuracy_score(y_test, y_pred)
    
    # Calculate ROC-AUC (requires binary or probability predictions)
    roc_auc = roc_auc_score(y_test, y_pred_prob[:, 1]) if y_pred_prob.shape[1] > 1 else roc_auc_score(y_test, y_pred_prob)
    
    # Generate the classification report
    report = classification_report(y_test, y_pred, target_names=class_names)
    
    return accuracy, roc_auc, report


In [46]:
y_test

array([0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1,
       1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1,
       1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1,
       1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0,
       1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0,
       0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1,
       0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1,
       1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [50]:
model_paths = ['Model/cnn_rnn_model.h5']
for path in model_paths:
    accuracy, roc_auc, report = evaluate_model(path, X_test, y_test)
    print(f"Modelo: {path}")
    print(f"Acurácia: {accuracy:.4f}")
    print(f"ROC-AUC: {roc_auc:.4f}")
    print("Relatório de Classificação:")
    print(report)

8/8 ━━━━━━━━━━━━━━━━━━━━ 12s 697ms/step
Modelo: Model/cnn_rnn_model.h5
Acurácia: 0.8714
ROC-AUC: 0.8087
Relatório de Classificação:
              precision    recall  f1-score   support

     Class 0       0.90      0.64      0.75        73
     Class 1       0.86      0.97      0.91       168

    accuracy                           0.87       241
   macro avg       0.88      0.81      0.83       241
weighted avg       0.87      0.87      0.86       241

